# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [29]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from requests.exceptions import SSLError
# Import API key
from api_keys import geoapify_key

In [30]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tidjikja,18.5564,-11.4271,39.00,10,4,5.27,MR,1712858280
1,1,puerto natales,-51.7236,-72.4875,10.25,57,0,2.06,CL,1712857820
2,2,nuuk,64.1835,-51.7216,-2.29,86,14,1.79,GL,1712858281
3,3,kamphaeng phet,16.2500,99.5000,30.17,46,9,1.02,TH,1712858281
4,4,tual,-5.6667,132.7500,28.46,67,69,1.90,ID,1712858281


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [31]:
# Configure the map plot
city_data_map = city_data_df.hvplot.points(x="Lng", y="Lat", size="Humidity", color="City", alpha=0.7,
                                        xlabel="x", ylabel="y", geo = True,
                                        tiles ="OSM",
                                       frame_width=700, frame_height=500)


# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[(city_data_df["Max Temp"] >= 20) & (city_data_df["Max Temp"] <= 25)] 

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,kiunga,-6.1219,141.2906,23.40,100,100,1.07,PG,1712858284
16,16,san mariano,7.5000,126.0000,21.32,91,70,1.16,PH,1712858284
25,25,katsuren-haebaru,26.3369,127.8719,21.01,93,0,1.54,JP,1712858286
28,28,adamstown,-25.0660,-130.1015,21.43,56,81,8.29,PN,1712858287
35,35,jamestown,42.0970,-79.2353,20.77,77,100,6.17,US,1712858015
...,...,...,...,...,...,...,...,...,...,...
571,571,napili-honokowai,20.9708,-156.6705,24.24,89,100,6.75,US,1712858376
572,572,gardnerville ranchos,38.8882,-119.7413,20.44,37,0,1.34,US,1712858376
574,574,kampene,-3.6000,26.6667,22.88,97,98,1.17,CD,1712858215
578,578,pitogo,13.7847,122.0881,23.07,90,100,2.54,PH,1712858377


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(filtered_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy())

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
14,kiunga,PG,-6.1219,141.2906,100,
16,san mariano,PH,7.5000,126.0000,91,
25,katsuren-haebaru,JP,26.3369,127.8719,93,
28,adamstown,PN,-25.0660,-130.1015,56,
35,jamestown,US,42.0970,-79.2353,77,
...,...,...,...,...,...,...
571,napili-honokowai,US,20.9708,-156.6705,89,
572,gardnerville ranchos,US,38.8882,-119.7413,37,
574,kampene,CD,-3.6000,26.6667,97,
578,pitogo,PH,13.7847,122.0881,90,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [34]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row["Lat"]
    longitude = row["Lng"]
    # Add filter and bias parameters with the current city"s latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError, SSLError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
   
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kiunga - nearest hotel: No hotel found
san mariano - nearest hotel: No hotel found
katsuren-haebaru - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
praia da vitoria - nearest hotel: Salles
wanning - nearest hotel: No hotel found
viqueque - nearest hotel: Hotel Wailakurini
hawaiian paradise park - nearest hotel: No hotel found
luanco - nearest hotel: Hotel Playa de Luanco
kwang binh - nearest hotel: Nhà hàng Amor Palace
quang ngai - nearest hotel: Ngoc Anh
kailua-kona - nearest hotel: Kona Seaside Hotel
labis - nearest hotel: No hotel found
nouadhibou - nearest hotel: Hotel valencia
fort bragg - nearest hotel: Airborne Inn Lodging
magole - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
deir ez-zor - nearest hotel: فندق السعيد
suva - nearest hotel: Extension 68
pontianak - nearest hotel: Golden Tulip Hotel
bubaque - nearest hotel: Chez Julio
haiku-pauwela - nearest hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
14,kiunga,PG,-6.1219,141.2906,100,No hotel found
16,san mariano,PH,7.5000,126.0000,91,No hotel found
25,katsuren-haebaru,JP,26.3369,127.8719,93,No hotel found
28,adamstown,PN,-25.0660,-130.1015,56,No hotel found
35,jamestown,US,42.0970,-79.2353,77,DoubleTree Jamestown
...,...,...,...,...,...,...
571,napili-honokowai,US,20.9708,-156.6705,89,Kahana Falls
572,gardnerville ranchos,US,38.8882,-119.7413,37,Holiday Inn
574,kampene,CD,-3.6000,26.6667,97,No hotel found
578,pitogo,PH,13.7847,122.0881,90,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(x="Lng", y="Lat", size="Humidity", hover_cols=["Hotel Name", "Country"], color="City", alpha=0.7,
                                        xlabel="x", ylabel="y", geo = True,
                                        tiles ="OSM",
                                       frame_width=700, frame_height=500)


# Display the map
hotel_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)